In [29]:
import nrrd
import os
import pandas as pd
import numpy as np
from scipy.ndimage import zoom
import csv
import matplotlib.pyplot as plt
from glob import glob
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, concatenate, add, Conv3DTranspose, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import backend as K
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model, load_model, save_model
from matplotlib.colors import ListedColormap
import Segment_support_lib

In [2]:
crop_ankle_train = 'F:\\Birva_Harsh\\Ankle_Crop_Data\\train\\Ankle'
crop_mask_train = 'F:\\Birva_Harsh\\Ankle_Crop_Data\\train\\Mask'
crop_ankle_test = 'F:\\Birva_Harsh\\Ankle_Crop_Data\\test\\Ankle'
crop_mask_test = 'F:\\Birva_Harsh\\Ankle_Crop_Data\\test\\Mask'

In [3]:
ankle_train = [os.path.join(crop_ankle_train, i) for i in os.listdir(crop_ankle_train)]
mask_train = [path.replace('\\Ankle\\', '\\Mask\\') for path in ankle_train]

ankle_test = [os.path.join(crop_ankle_test, i) for i in os.listdir(crop_ankle_test)]
mask_test = [path.replace('\\Ankle\\', '\\Mask\\') for path in ankle_test]

In [4]:
df_train = pd.DataFrame( data = {
    'ankle': ankle_train,
    'mask': mask_train
})

df_test = pd.DataFrame( data = {
    'ankle': ankle_test,
    'mask': mask_test
})

In [5]:
def load_and_preprocess_image(file_path,resize):
    img,info = nrrd.read(file_path)
    if resize == 128:
        img = img[::2,::2,::2]
    img = np.clip(img, a_min=-1024, a_max=None)
    img = img[:,:,:,np.newaxis]   
    img_scaled = img.astype(np.float32) / 1024.0
    return img_scaled

In [6]:
def load_and_preprocess_mask(mask_path,img_path,resize):
    img,hinfo = nrrd.read(img_path)
    mask,minfo = nrrd.read(mask_path)
    
    if resize == 128:
        img = img[::2,::2,::2]
        mask = mask[::2,::2,::2]
        
    mask = np.where(mask==3,1,0) 
    thres = np.where(img < 200, 0, 1)
    res = thres-mask
    res = np.where(res<=0,0,1)
    
    mask = np.stack((mask,res), axis=-1)
    mask = mask.astype(np.float32)
    
    return mask

In [7]:
def generate_data_generator(dataframe,resize, batch_size=1):
    num_samples = len(dataframe)
    while True:
        indices = np.random.randint(0, num_samples, batch_size)
        batch_ankle = []
        batch_mask = []
        for idx in indices:
            ankle_img = load_and_preprocess_image(dataframe.iloc[idx]['ankle'],resize)
            mask_img = load_and_preprocess_mask(dataframe.iloc[idx]['mask'],dataframe.iloc[idx]['ankle'],resize)
            batch_ankle.append(ankle_img)
            batch_mask.append(mask_img)
        batch_ankle = np.array(batch_ankle)
        batch_mask = np.array(batch_mask)
        
        yield batch_ankle, batch_mask

In [8]:
batch_size = 1
train_generator = generate_data_generator(df_train, batch_size=batch_size,resize=256)
test_generator = generate_data_generator(df_test, batch_size=batch_size,resize=256)

In [9]:
def dice_coeff(y_true, y_pred, smooth=100):
    y_true_flatten = K.flatten(y_true)
    y_pred_flatten = K.flatten(y_pred)

    intersection = K.sum(y_true_flatten * y_pred_flatten)
    union = K.sum(y_true_flatten) + K.sum(y_pred_flatten)
    return (2 * intersection + smooth) / (union + smooth)

def dice_coeff_loss(y_true, y_pred, smooth=100):
    return 1 - dice_coeff(y_true, y_pred, smooth)

In [10]:
def DICE_COE(mask1, mask2):
    intersect = np.sum(mask1*mask2)
    fsum = np.sum(mask1)
    ssum = np.sum(mask2)
    dice = (2 * intersect ) / (fsum + ssum)
    dice = np.mean(dice)
    dice = round(dice, 3)
    return dice    

In [11]:
def encoder_block(inputs, filters):
    conv = Conv3D(filters, kernel_size = (3,3,3), padding="same", activation = 'relu', kernel_initializer='he_uniform')(inputs)
    conv = Conv3D(filters, kernel_size = (3,3,3), padding="same", activation = 'relu', kernel_initializer='he_uniform')(conv)
    x = BatchNormalization()(conv)
    pool = MaxPooling3D(pool_size=(2,2,2), padding="same")(x)
    
    return pool,x

def decoder_block(inputs, filters, x):
    up = Conv3DTranspose(filters, kernel_size=(3,3,3), strides=(2, 2, 2), padding="same", kernel_initializer='he_uniform')(inputs)
    con = concatenate([up, x])
    conv = Conv3D(filters, kernel_size = (3,3,3), padding="same", activation = 'relu', kernel_initializer='he_uniform')(con)
    conv = Conv3D(filters, kernel_size = (3,3,3), padding="same", activation = 'relu', kernel_initializer='he_uniform')(conv)
    x = BatchNormalization()(conv)
    
    return x

In [12]:
f = 16

In [25]:
def Pocketnet_16(input_shape = (None, None, None, 1)):
    inputs = Input(input_shape)
    x = Conv3D(filters = f, kernel_size = (3,3,3), padding = 'same', activation = 'relu', kernel_initializer='he_uniform')(inputs)
    
    b1,c1 = encoder_block(x, f)
    b2,c2 = encoder_block(b1, f*2)
    b3,c3 = encoder_block(b2, f*2)
    b4,c4 = encoder_block(b3, f*2)

    conv = Conv3D(f*2, kernel_size = (3,3,3), padding="same", activation = 'relu', kernel_initializer='he_uniform')(b4)
    conv = Conv3D(f*2, kernel_size = (3,3,3), padding="same", activation = 'relu', kernel_initializer='he_uniform')(conv)

    b4 = decoder_block(conv, f, c4)
    b5 = decoder_block(b4, f*2, c3)
    b6 = decoder_block(b5, f*2, c2)
    b7 = decoder_block(b6, f*2, c1)
    
    outputs = Conv3D(filters = 2, kernel_size = (1,1,1), activation = "sigmoid")(b7)
    
    return Model(inputs=[inputs], outputs = [outputs])

In [26]:
filepath = "F:\\Birva_Harsh\\Ankle_models\\pocketnet_256_32.keras"

checkpoint = ModelCheckpoint(filepath, 
                             verbose=1 
                             )

In [27]:
pocket_16 = Pocketnet_16(input_shape = (None,None,None,1))
pocket_16.compile(optimizer='adam', loss=dice_coeff_loss, metrics=[dice_coeff])

In [28]:
epochs = 50

In [29]:
history = pocket_16.fit(
        train_generator,
        steps_per_epoch=len(df_train) // batch_size,
        epochs=epochs,
        callbacks=[checkpoint],
        validation_data=test_generator,
        validation_steps=len(df_test) // batch_size
    )

Epoch 1/50


ResourceExhaustedError: Graph execution error:

Detected at node 'gradient_tape/model_1/batch_normalization_17/FusedBatchNormGradV3' defined at (most recent call last):
    File "C:\Users\Admin\.conda\envs\harsh_birva\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\Admin\.conda\envs\harsh_birva\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\Admin\.conda\envs\harsh_birva\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "C:\Users\Admin\.conda\envs\harsh_birva\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "C:\Users\Admin\.conda\envs\harsh_birva\lib\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "C:\Users\Admin\.conda\envs\harsh_birva\lib\site-packages\tornado\platform\asyncio.py", line 205, in start
      self.asyncio_loo

# Half U-Net

In [ ]:
f = 16

In [ ]:
def Unet_8(input_shape = (None, None, None, 1)):
    inputs = Input(input_shape)
    x = Conv3D(filters = f, kernel_size = (3,3,3), padding = 'same', activation = 'relu', kernel_initializer='he_uniform')(inputs)
    x = BatchNormalization()(x)
    
    b1,c1 = encoder_block(x, f)
    b2,c2 = encoder_block(b1, f)
    b3,c3 = encoder_block(b2, f*2)
    b4,c4 = encoder_block(b3, f*2)

    conv = Conv3D(f*4, kernel_size = (3,3,3), padding="same", activation = 'relu', kernel_initializer='he_uniform')(b4)
    conv = Conv3D(f*4, kernel_size = (3,3,3), padding="same", activation = 'relu', kernel_initializer='he_uniform')(conv)

    b4 = decoder_block(conv, f*2, c4)
    b5 = decoder_block(b4, f*2, c3)
    b6 = decoder_block(b5, f, c2)
    b7 = decoder_block(b6, f, c1)
    
    outputs = Conv3D(filters = 2, kernel_size = (1,1,1), activation = "sigmoid")(b7)
    
    return Model(inputs=[inputs], outputs = [outputs])

In [ ]:
filepath = "F:\\Birva_Harsh\\Ankle_models\\half_unet_256_8.keras"

checkpoint = ModelCheckpoint(filepath, 
                             verbose=1 
                             )

In [ ]:
unet_8 = Unet_8(input_shape = (None,None,None,1))
unet_8.compile(optimizer='adam', loss=dice_coeff_loss, metrics=[dice_coeff])

In [ ]:
epochs = 50

In [ ]:
history = unet_8.fit(
        train_generator,
        steps_per_epoch=len(df_train) // batch_size,
        epochs=epochs,
        callbacks=[checkpoint],
        validation_data=test_generator,
        validation_steps=len(df_test) // batch_size
    )

# U-Net

In [ ]:
f = 8

In [ ]:
def Unet_8(input_shape = (None, None, None, 1)):
    inputs = Input(input_shape)
    x = Conv3D(filters = f, kernel_size = (3,3,3), padding = 'same', activation = 'relu', kernel_initializer='he_uniform')(inputs)
    x = BatchNormalization()(x)
    
    b1,c1 = encoder_block(x, f)
    b2,c2 = encoder_block(b1, f*2)
    b3,c3 = encoder_block(b2, f*4)
    b4,c4 = encoder_block(b3, f*8)

    conv = Conv3D(f*8, kernel_size = (3,3,3), padding="same", activation = 'relu', kernel_initializer='he_uniform')(b4)
    conv = Conv3D(f*8, kernel_size = (3,3,3), padding="same", activation = 'relu', kernel_initializer='he_uniform')(conv)

    b4 = decoder_block(conv, f*8, c4)
    b5 = decoder_block(b4, f*4, c3)
    b6 = decoder_block(b5, f*2, c2)
    b7 = decoder_block(b6, f, c1)
    
    outputs = Conv3D(filters = 2, kernel_size = (1,1,1), activation = "sigmoid")(b7)
    
    return Model(inputs=[inputs], outputs = [outputs])

In [ ]:
filepath = "F:\\Birva_Harsh\\Ankle_models\\unet_256_8.keras"

checkpoint = ModelCheckpoint(filepath, 
                             verbose=1 
                             )

In [ ]:
unet_8 = Unet_8(input_shape = (None,None,None,1))
unet_8.compile(optimizer='adam', loss=dice_coeff_loss, metrics=[dice_coeff])

In [ ]:
epochs = 50

In [ ]:
history = unet_8.fit(
        train_generator,
        steps_per_epoch=len(df_train) // batch_size,
        epochs=epochs,
        callbacks=[checkpoint],
        validation_data=test_generator,
        validation_steps=len(df_test) // batch_size
    )

# Resnet Type Net (Filters=16)

In [30]:
def encoder_block(inputs, filters):
    conv = Conv3D(filters = f, kernel_size = (3,3,3), padding = 'same', activation = 'relu', kernel_initializer='he_uniform')(inputs)
    conv = Conv3D(filters = f, kernel_size = (3,3,3), padding = 'same', activation = 'relu', kernel_initializer='he_uniform')(conv)
    pool = MaxPooling3D(pool_size=(2,2,2), padding="same")(conv)

    return pool,conv

def decoder_block(inputs, filters, x):
    up = Conv3DTranspose(filters, kernel_size=(3,3,3), strides=(2, 2, 2), padding="same", kernel_initializer='he_uniform')(inputs)
    conv = Conv3D(filters = f, kernel_size = (3,3,3), padding = 'same', activation = 'relu', kernel_initializer='he_uniform')(up)
    conv = Conv3D(filters = f, kernel_size = (3,3,3), padding = 'same', activation = 'relu', kernel_initializer='he_uniform')(conv)

    if x is not None:
        x = BatchNormalization()(x)
        conv = BatchNormalization()(conv)
        conv = add([x, conv])

    return conv

In [31]:
def u_block(inputs, filters):
    b1,c1 = encoder_block(inputs, filters)
    b2,c2 = encoder_block(b1, filters)

    conv = Conv3D(filters = f, kernel_size = (3,3,3), padding = 'same', activation = 'relu', kernel_initializer='he_uniform')(b2)
    conv = Conv3D(filters = f, kernel_size = (3,3,3), padding = 'same', activation = 'relu', kernel_initializer='he_uniform')(conv)

    b4 = decoder_block(conv, filters,None)
    b5 = decoder_block(b4, filters, c1)

    return b5

In [32]:
f = 16
num_blocks = 4

In [33]:
def res_type_16(input_shape = (None, None, None, 1)):

    inputs = Input(input_shape)
    x = Conv3D(filters = f, kernel_size = (3,3,3), padding = 'same', activation = 'relu', kernel_initializer='he_uniform')(inputs)
    x = BatchNormalization()(x)

    for i in range(num_blocks):
        x = u_block(x,f)
        b = Conv3D(filters = f, kernel_size = (3,3,3), padding = 'same', activation = 'relu', kernel_initializer='he_uniform')(inputs)
        b = BatchNormalization()(b)
        x = add([x, b])
    
    outputs = Conv3D(filters = 2, kernel_size = (1,1,1), activation = "sigmoid")(x)
    
    return Model(inputs=[inputs], outputs = [outputs])

In [34]:
epochs=50

In [35]:
filepath = "F:\\Birva_Harsh\\Ankle_models\\restype_16.keras"

checkpoint = ModelCheckpoint(filepath, 
                             verbose=1 
                             )

1/1 [==============================] - 0s 26ms/step


1/1 [==============================] - 0s 26ms/step
Epoch 1/50
75/75 [==============================] - ETA: 0s - loss: 0.5779 - dice_coeff: 0.4221
Epoch 1: saving model to G:\Harsh_Birva_minor_Proj\new_retina_models_5\restype_16-2.keras
75/75 [==============================] - 110s 1s/step - loss: 0.5779 - dice_coeff: 0.4221 - val_loss: 0.9847 - val_dice_coeff: 0.0153
Epoch 2/50
75/75 [==============================] - ETA: 0s - loss: 0.2591 - dice_coeff: 0.7409
Epoch 2: saving model to G:\Harsh_Birva_minor_Proj\new_retina_models_5\restype_16-2.keras
75/75 [==============================] - 120s 2s/step - loss: 0.2591 - dice_coeff: 0.7409 - val_loss: 0.9955 - val_dice_coeff: 0.0045
Epoch 3/50
75/75 [==============================] - ETA: 0s - loss: 0.1768 - dice_coeff: 0.8232
Epoch 3: saving model to G:\Harsh_Birva_minor_Proj\new_retina_models_5\restype_16-2.keras
75/75 [==============================] - 107s 1s/step - loss: 0.1768 - dice_coeff: 0.8232 - val_loss: 0.9955 - val_dice_co

75/75 [==============================] - 105s 1s/step - loss: 0.1032 - dice_coeff: 0.8968 - val_loss: 0.1515 - val_dice_coeff: 0.8485
Epoch 21/50
75/75 [==============================] - ETA: 0s - loss: 0.1064 - dice_coeff: 0.8936
Epoch 21: saving model to G:\Harsh_Birva_minor_Proj\new_retina_models_5\restype_16-2.keras
75/75 [==============================] - 120s 2s/step - loss: 0.1064 - dice_coeff: 0.8936 - val_loss: 0.1337 - val_dice_coeff: 0.8663
Epoch 22/50
75/75 [==============================] - ETA: 0s - loss: 0.0999 - dice_coeff: 0.9001
Epoch 22: saving model to G:\Harsh_Birva_minor_Proj\new_retina_models_5\restype_16-2.keras
75/75 [==============================] - 120s 2s/step - loss: 0.0999 - dice_coeff: 0.9001 - val_loss: 0.1461 - val_dice_coeff: 0.8539
Epoch 23/50
75/75 [==============================] - ETA: 0s - loss: 0.1009 - dice_coeff: 0.8991
Epoch 23: saving model to G:\Harsh_Birva_minor_Proj\new_retina_models_5\restype_16-2.keras
75/75 [===========================

75/75 [==============================] - ETA: 0s - loss: 0.0963 - dice_coeff: 0.9037
Epoch 46: saving model to G:\Harsh_Birva_minor_Proj\new_retina_models_5\restype_16-2.keras
75/75 [==============================] - 106s 1s/step - loss: 0.0963 - dice_coeff: 0.9037 - val_loss: 0.1187 - val_dice_coeff: 0.8813
Epoch 47/50
75/75 [==============================] - ETA: 0s - loss: 0.0962 - dice_coeff: 0.9038
Epoch 47: saving model to G:\Harsh_Birva_minor_Proj\new_retina_models_5\restype_16-2.keras
75/75 [==============================] - 120s 2s/step - loss: 0.0962 - dice_coeff: 0.9038 - val_loss: 0.1201 - val_dice_coeff: 0.8799
Epoch 48/50
75/75 [==============================] - ETA: 0s - loss: 0.0938 - dice_coeff: 0.9062
Epoch 48: saving model to G:\Harsh_Birva_minor_Proj\new_retina_models_5\restype_16-2.keras
75/75 [==============================] - 106s 1s/step - loss: 0.0938 - dice_coeff: 0.9062 - val_loss: 0.1128 - val_dice_coeff: 0.8872
Epoch 49/50
75/75 [===========================

1/1 [==============================] - 0s 16ms/step
Epoch 1/50
75/75 [==============================] - ETA: 0s - loss: 0.5749 - dice_coeff: 0.4251
Epoch 1: saving model to G:\Harsh_Birva_minor_Proj\new_retina_models_5\restype_16-3.keras
75/75 [==============================] - 110s 1s/step - loss: 0.5749 - dice_coeff: 0.4251 - val_loss: 0.9793 - val_dice_coeff: 0.0207
Epoch 2/50
75/75 [==============================] - ETA: 0s - loss: 0.2726 - dice_coeff: 0.7274
Epoch 2: saving model to G:\Harsh_Birva_minor_Proj\new_retina_models_5\restype_16-3.keras
75/75 [==============================] - 120s 2s/step - loss: 0.2726 - dice_coeff: 0.7274 - val_loss: 0.9976 - val_dice_coeff: 0.0024
Epoch 3/50
75/75 [==============================] - ETA: 0s - loss: 0.2105 - dice_coeff: 0.7895
Epoch 3: saving model to G:\Harsh_Birva_minor_Proj\new_retina_models_5\restype_16-3.keras
75/75 [==============================] - 120s 2s/step - loss: 0.2105 - dice_coeff: 0.7895 - val_loss: 0.9937 - val_dice_co

...

1/1 [==============================] - 0s 46ms/step
Epoch 1/50
75/75 [==============================] - ETA: 0s - loss: 0.5442 - dice_coeff: 0.4558
Epoch 1: saving model to G:\Harsh_Birva_minor_Proj\new_retina_models_5\restype_16-5.keras
75/75 [==============================] - 126s 2s/step - loss: 0.5442 - dice_coeff: 0.4558 - val_loss: 0.9679 - val_dice_coeff: 0.0321
Epoch 2/50
75/75 [==============================] - ETA: 0s - loss: 0.2405 - dice_coeff: 0.7595
Epoch 2: saving model to G:\Harsh_Birva_minor_Proj\new_retina_models_5\restype_16-5.keras


75/75 [==============================] - 106s 1s/step - loss: 0.2405 - dice_coeff: 0.7595 - val_loss: 0.9941 - val_dice_coeff: 0.0059
Epoch 3/50
75/75 [==============================] - ETA: 0s - loss: 0.1777 - dice_coeff: 0.8223
Epoch 3: saving model to G:\Harsh_Birva_minor_Proj\new_retina_models_5\restype_16-5.keras
75/75 [==============================] - 120s 2s/step - loss: 0.1777 - dice_coeff: 0.8223 - val_loss: 0.9967 - val_dice_coeff: 0.0033
Epoch 4/50
75/75 [==============================] - ETA: 0s - loss: 0.1552 - dice_coeff: 0.8448
Epoch 4: saving model to G:\Harsh_Birva_minor_Proj\new_retina_models_5\restype_16-5.keras
75/75 [==============================] - 106s 1s/step - loss: 0.1552 - dice_coeff: 0.8448 - val_loss: 0.9919 - val_dice_coeff: 0.0081
Epoch 5/50
75/75 [==============================] - ETA: 0s - loss: 0.1357 - dice_coeff: 0.8643
Epoch 5: saving model to G:\Harsh_Birva_minor_Proj\new_retina_models_5\restype_16-5.keras
75/75 [==============================] -

75/75 [==============================] - ETA: 0s - loss: 0.0978 - dice_coeff: 0.9022
Epoch 28: saving model to G:\Harsh_Birva_minor_Proj\new_retina_models_5\restype_16-5.keras
75/75 [==============================] - 105s 1s/step - loss: 0.0978 - dice_coeff: 0.9022 - val_loss: 0.1297 - val_dice_coeff: 0.8703
Epoch 29/50
75/75 [==============================] - ETA: 0s - loss: 0.0993 - dice_coeff: 0.9007
Epoch 29: saving model to G:\Harsh_Birva_minor_Proj\new_retina_models_5\restype_16-5.keras
75/75 [==============================] - 120s 2s/step - loss: 0.0993 - dice_coeff: 0.9007 - val_loss: 0.1205 - val_dice_coeff: 0.8795
Epoch 30/50
75/75 [==============================] - ETA: 0s - loss: 0.0996 - dice_coeff: 0.9004
Epoch 30: saving model to G:\Harsh_Birva_minor_Proj\new_retina_models_5\restype_16-5.keras
75/75 [==============================] - 120s 2s/step - loss: 0.0996 - dice_coeff: 0.9004 - val_loss: 0.1315 - val_dice_coeff: 0.8685
Epoch 31/50
75/75 [===========================

1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 14ms/step


In [36]:
restype_16 = res_type_16(input_shape = (None,None,None,1))
restype_16.compile(optimizer='adam', loss=dice_coeff_loss, metrics=[dice_coeff])

In [37]:
history = restype_16.fit(
        train_generator,
        steps_per_epoch=len(df_train) // batch_size,
        epochs=epochs,
        callbacks=[checkpoint],
        validation_data=test_generator,
        validation_steps=len(df_test) // batch_size
    )

Epoch 1/50


ResourceExhaustedError: Graph execution error:

Detected at node 'model/batch_normalization_8/FusedBatchNormV3' defined at (most recent call last):
    File "C:\Users\Admin\.conda\envs\harsh_birva\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\Admin\.conda\envs\harsh_birva\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\Admin\.conda\envs\harsh_birva\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "C:\Users\Admin\.conda\envs\harsh_birva\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "C:\Users\Admin\.conda\envs\harsh_birva\lib\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "C:\Users\Admin\.conda\envs\harsh_birva\lib\site-packages\tornado\platform\asyncio.py", line 205, in start
      self.asyncio_loop.run_forever()
    F